In [43]:
import cv2
import pytesseract
import numpy as np
from pytesseract import Output
from PIL import Image
  
#open image in png format
# img_png = Image.open('./Images/data4.png')
# img_png = img_png.convert('RGB') 
# #The image object is used to save the image in jpg format
# img_png.save('./Images/data8.jpg') 
img_source = cv2.imread('./Images/data5.jpg')
# d = pytesseract.image_to_data(img, output_type=Output.DICT)
# print(d)
kernel = np.ones ((5,5), np.uint8)

gray_scaled1 = cv2.cvtColor(img_source, cv2.COLOR_BGR2GRAY)
gray_image = 255 - gray_scaled1
# print(np.count_nonzero(gray_scaled1==255))
img_er = cv2.erode(gray_image, kernel, iterations=4)
img_di = cv2.dilate(gray_image, kernel, iterations=2)
# cv2.imshow('Erosion', img_er)
# cv2.imshow('Dilation', img_di)

ret,processed_img = cv2.threshold(np.array(img_di), 125, 255, cv2.THRESH_BINARY)
# cv2.imshow('Processed', processed_img)
# cv2.waitKey(0)
config = ('-l eng --psm 13')
h = pytesseract.image_to_string(255 - processed_img, config=config )#"--psm 7 outputbase digits")
print(h)
s = pytesseract.image_to_string(255 - processed_img, config="--psm 7 outputbase digits")
print(s)

.




In [12]:
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
 
 
def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
 
 
def opening(image):
    kernel = np.ones((5, 5), np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)
 
 
def canny(image):
    return cv2.Canny(image, 100, 200)
 
 
gray = get_grayscale(img_source)
thresh = thresholding(gray)
opening = opening(gray)
canny = canny(gray)
 
for img in [img_source, gray, thresh, opening, canny]:
    d = pytesseract.image_to_data(img, output_type=Output.DICT)
    n_boxes = len(d['text'])
 
    # back to RGB
    if len(img.shape) == 2:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
 
    for i in range(n_boxes):
        if int(d['conf'][i]) > 60:
            (text, x, y, w, h) = (d['text'][i], d['left'][i], d['top'][i], d['width'][i], d['height'][i])
            # don't show empty text
            if text and text.strip() != "":
                img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
                img = cv2.putText(img, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)
 
    cv2.imshow('img', img)
    cv2.waitKey(0)